# Scraping the website


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(options=options)

In [2]:
from time import sleep
wd.implicitly_wait(30)
wd.get("https://dework.xyz/")
bounties_button=wd.find_element(By.CLASS_NAME,'ant-btn-default')
wd.get(bounties_button.get_attribute("href"))

In [ ]:
wd.find_element(By.CSS_SELECTOR,"div#rc-tabs-0-tab-organizations").click()

In [ ]:
data_communities=[]
def collections_to_dic(DAOS):
    for result in DAOS:
        DA0_link=result.find_element(By.TAG_NAME,"a").get_attribute("href")
        DA0_name=result.find_element(By.TAG_NAME,"h3").text
        data_communities.append({"DAO":DA0_name,"link":DA0_link})


In [ ]:
pages=wd.find_elements(By.CSS_SELECTOR,"li.ant-pagination-item")
page_numbers=[int(page.get_attribute("title")) for page in pages]
for page in range(1,max(page_numbers)+1):
    print(page)
    wd.get("https://app.dework.xyz/?page="+str(page))
    #wd.find_element(By.CSS_SELECTOR,"li[title='{}']".format(page))
    collections=wd.find_elements(By.CLASS_NAME,"ant-table-row")
    collections_to_dic(collections)
            

In [ ]:
for i in data_communities:
    try:
      i['task_id'].remove("TODO:default")
    except:
      pass 
    try:
       i['task_id'].remove("IN_PROGRESS:default")
    except:
       pass 
    try:
       i['task_id'].remove("DONE:default")
    except:
    try:
       i['task_id'].remove("DONE:default")
      pass 

In [ ]:
#to collect the tasks iD
wd.implicitly_wait(40)
for DAO in data_communities:
     print(DAO["DAO"])
     wd.get(DAO["link"]+"/board")
     scroll_screens=wd.find_elements(By.CSS_SELECTOR,"div.ReactVirtualized__Grid")
     #scroll_horizontal=wd.find_element(By.CSS_SELECTOR,"div.ant-row.dewo-task-board")
     DAO["task_id"]=[0]
     for idx,table in enumerate(wd.find_elements(By.CSS_SELECTOR,".ReactVirtualized__Grid__innerScrollContainer")):
          while True:
              add=0
              tasks=table.find_elements(By.CSS_SELECTOR,"div[style^='height:']")
              
              for task in tasks:
                          try:
                            id=task.get_attribute("data-rbd-draggable-id")
                          except:
                            id=0
                          if id not in DAO["task_id"]:
                                    DAO["task_id"].append(id)
                                    add+=1
              try:
                scroll_screens[idx].send_keys(Keys.PAGE_DOWN)
              except:
                pass
              sleep(3)
              tables=wd.find_elements(By.CSS_SELECTOR,".ReactVirtualized__Grid__innerScrollContainer")
              table=tables[idx]
              if add==0:
                  scroll_screens=wd.find_elements(By.CSS_SELECTOR,"div.ReactVirtualized__Grid")
                  if idx==len(tables)-2:
                          sleep(3)
                          wd.execute_script("""
                          var elements = document.querySelectorAll('.ant-space-item');
                          elements[elements.length-1].scrollIntoView();
                                                                         """)
                  #scroll_horizontal.send_keys(Keys.PAGE_DOWN)
                  break 
     print(len(DAO["task_id"]))
    


In [ ]:
remove_list=[0,'TODO:default','IN_PROGRESS:default',"IN_REVIEW:default",None,"DONE:paid","DONE:processing-payment","DONE:needs_payment","DONE:needs-payment"]
DOA_without_tasks=[]
for i in data_communities:
    if "task_id" in i.keys():
         for id in i["task_id"]:
                 if id in remove_list:
                         i["task_id"].remove(id)
         if len(i["task_id"])==0:
                 del i["task_id"]
                 DOA_without_tasks.append(i["DAO"])
                 
         
                         
       

In [ ]:
remove_list=[0,'TODO:default','IN_PROGRESS:default',"IN_REVIEW:default",None,"DONE:paid","DONE:processing-payment","DONE:needs_payment","DONE:needs-payment","DONE:default"]
DOA_without_tasks=[]
for i in dic_task_id:
    if "task_id" in i.keys():
         for id in i["task_id"]:
                 if id in remove_list:
                         i["task_id"].remove(id)
         if len(i["task_id"])==0:
                 del i["task_id"]
                 DOA_without_tasks.append(i["DAO"])

In [ ]:
DOA_without_tasks
dic_task_id=[i for i in data_communities if "task_id" in i.keys()]

In [ ]:
print(dic_task_id)
sum([len(i['task_id']) for i in dic_task_id])

In [ ]:
for doa in dic_task_id:
    print(len(doa['task_id']))
    print(doa['task_id'])
    for i in doa['task_id']:
        if i in remove_list:
            doa['task_id'].remove(i)
        if len(i)<20:
            print(i)
            doa['task_id'].remove(i)


In [ ]:
print([i['task_id'] for i in dic_task_id])

In [2]:
#!pip install openpyxl
a=pd.read_csv("id.csv")
a["task_id"]=a["task_id"].map(lambda a:a.lstrip("['").rstrip("']"))
dic_task_id=a.to_dict("records")

In [3]:
#for data in dic_task_id:
      #data["task_id"]=data["task_id"].replace("['","")
      #data["task_id"]=data["task_id"].replace("']","")
for data in dic_task_id:
       data["task_id"]=data["task_id"].split(",")
       #for i in data["task_id"]:
              #i=i.replace(" ","")
              #i=i.replace("'","")
              #print(i)
              
       #print(data["task_id"])


In [4]:
new=[]
for data in dic_task_id:
     new_data={}
     new_data["DAO"]=data["DAO"]
     new_data["task_id"]=[]
     for i in data["task_id"]:
          new_data["task_id"].append(i.lstrip(" '").rstrip("'"))
     new_data["link"]=data["link"]
     new.append(new_data)

In [5]:
remove_list=[0,'TODO:default','IN_PROGRESS:default',"IN_REVIEW:default",None,"DONE:paid","DONE:processing-payment","DONE:needs_payment","DONE:needs-payment","DONE:default"]
for doa in new:
  for i in doa['task_id']:
        if i in remove_list:
            doa['task_id'].remove(i)
        

In [6]:
for i in new:
    if i["DAO"]=="Kukuza":
       IND=new.index(i)
       IND_2=i["task_id"].index("fec8cb43-105c-4060-9652-449ffdd532d2")
    

In [ ]:
import json
#import multiprocessing
from selenium.common.exceptions import TimeoutException
from multiprocessing import Pool, cpu_count
import threading 
#Diamante Luz
tasks=[]
class ScrapeThread(threading.Thread):
    def __init__(self, url):
        threading.Thread.__init__(self)
        self.url = url
  
    def run(self):
        print("start")
        wd=webdriver.Chrome(options=options)
        wd.maximize_window()
        wd.get(self.url)
        try:
           WebDriverWait(wd, timeout=40).until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.mb-7")))
        except TimeoutException:
           print(self.url)
        try:DAO=wd.find_element(By.CSS_SELECTOR,"a.ant-breadcrumb-link").get_property("innerText")
        except:DAO=None 
        try:labels=wd.find_element(By.CSS_SELECTOR,"div.ant-row[style='column-gap: 8px;']").get_property("innerText")
        except:labels=None
        try:task_title=wd.find_element(By.CSS_SELECTOR,"div.ant-form-item-control-input textarea#Task\ Form\ \(update\)_name").get_property("textContent")
        except:task_title=None
        try:due_date=wd.find_element(By.CSS_SELECTOR,"span.ant-tag:has(> span.anticon-calendar[role='img'][aria-label='calendar'])").get_property("innerText")
        except:due_date=None
        try:wd.find_element(By.CSS_SELECTOR,"div.mb-7 button strong").click()
        except:pass
        try:task_description=wd.find_element(By.CSS_SELECTOR,"div.mb-7").get_property("innerText")
        except:task_description=None
        #priority=wd.find_element(By.CSS_SELECTOR,"div.ant-row.ant-form-item.has(> label[for='Task Form (update)_priority'])*::text")
        #priority=None 
        task_activities=wd.find_elements(By.CSS_SELECTOR,"div.ant-timeline-item-content div.ant-row.ant-row-middle[role='row']")
        activities=[]
        time_stamps=[]
        try:bounties=wd.find_element(By.CSS_SELECTOR,"span.ant-tag[style='background-color: white; color: black; min-width: 0px; overflow: hidden; text-overflow: ellipsis;']").get_property("innerText")
        except: bounties=None
      
         
        try:reviwers=wd.find_element(By.CSS_SELECTOR,"div.ant-row.ant-form-item:has( label[for='Task Form (update)_ownerIds'])").find_element(By.CSS_SELECTOR,"div:nth-child(2)").get_property("innerText")   
        except:reviwers=None 
        try:priority=wd.find_element(By.CSS_SELECTOR,"div.ant-row.ant-form-item:has( label[for='Task Form (update)_priority'])").find_element(By.CSS_SELECTOR,"div:nth-child(2)").get_property("innerText")
        except:priority=None  
        try:activity_comments=[i.text for i in wd.find_elements(By.CSS_SELECTOR,"div.ant-timeline-item-content div.ProseMirror[role='textbox'][contenteditable='false'] > p")]
        except:activity_comments=None
        for activity in task_activities:
                                 #print(activity.text+"/")
            try:activities.append(activity.find_element(By.CLASS_NAME,"ant-typography").get_property("innerText"))
            except:pass
            try:time_stamps.append(activity.find_element(By.CLASS_NAME,"ant-typography-secondary").get_property("innerText"))
            except:pass
        try:subtasks=[i.get_property("innerText") for i in wd.find_elements(By.CSS_SELECTOR,"td.ant-table-cell.w-full > div.ant-typography")]
        except:subtasks=None 
        try:subtasks_id=[i.get_attribute("data-row-key") for i in wd.find_elements(By.CSS_SELECTOR,"tbody.ant-table-tbody > tr[data-row-key][index]")]
        except:subtasks_id=None
        try: tags=[i.find_element(By.CSS_SELECTOR,"span.ant-tag").get_property("innerText") for i in wd.find_elements(By.CSS_SELECTOR,"div.ant-select-selection-overflow-item > span")]
        except:tags=None
        tasks.append({"due_date":due_date,"activity_comments":activity_comments,"link":self.url,"labels":labels,"tags":tags,"subtasks_id":subtasks_id,"reviewers":reviwers,"DAO":DAO,"time_stamps":time_stamps,"activities": activities,"subtasks":subtasks,"priority":priority,"task_title":task_title,"task_description":task_description,"subtasks":subtasks,"bounties":bounties})
        print("success")
        wd.close()
def batch_indices(links,size=10):
     if len(links)<size:
          return([0])
     else:
          return(range(0,len(links),size))
     
    
processing_size=5
Number=0
for DAO in new[IND:]:
          print(DAO["DAO"])
          if DAO["DAO"]=="Kukuza":
               lists=batch_indices(DAO["task_id"][IND_2],processing_size)
          else:
               lists=batch_indices(DAO["task_id"],processing_size)
          for i in lists:
               if i+processing_size<len(DAO["task_id"]):
                       links=DAO["task_id"][i:i+processing_size]
               else:
                       links=DAO["task_id"][i:]
               threads = []
               tasks=[]    
               for link in links:
                       url=DAO["link"]+"/board?taskId="+link
                       t = ScrapeThread(url)
                       t.start()
                       threads.append(t)
  
               for t in threads:
                       t.join()
               Number=Number+len(tasks)
               #print("percentage_done:{}".format(Number/len(urls)))
               with open('data.json', 'a') as f:
                       json.dump(tasks, f)
                       f.close()
          
         

        
          

In [ ]:
wd.find_element(By.CSS_SELECTOR,"div.ant-row.ant-form-item:has( label[for='Task Form (update)_ownerIds'])").find_element(By.CSS_SELECTOR,"div:nth-child(2)").get_attribute("textContent")

In [ ]:
wd.get("https://app.dework.xyz/nation3/board?taskId=92d1288a-c60c-4547-93c0-f724e7edd81c")
try:wd.find_elements(By.CSS_SELECTOR,"div.mb-7 button strong").click()
except:pass
print(wd.find_element(By.CSS_SELECTOR,"div.mb-7").get_property("innerText"))

In [ ]:
dic_task_id
df=pd.DataFrame.from_records(dic_task_id)
df.to_csv("id.csv")

In [ ]:
list(tasks[0].keys())

In [ ]:
#get the data 
data=pd.read_csv("data.csv",low_memory=False)
data.columns=list(tasks[0].keys())
data=data.drop_duplicates()

In [ ]:
        
        webdriver.Chrome(options=options)
        wd.maximize_window()
        wd.implicitly_wait(40)
        wd.get(self.url)
        print(self.url)
        try:labels=wd.find_element(By.CSS_SELECTOR,"div.ant-row[style='column-gap: 8px;']").get_attribute("innerHTML")
        except:labels=None
        try:task_title=wd.find_element(By.CSS_SELECTOR,"div.ant-form-item-control-input #Task\ Form\ \(update\)_name").get_attribute("textContent")
        except:task_title=None
        try:wd.find_elements(By.CSS_SELECTOR,"div.mb-7 > div[class^='ViewMoreContainer']+button").click()
        except:pass
        try:task_description=wd.find_element(By.CSS_SELECTOR,"div.mb-7").get_attirbute("textContent")
        except:task_description=None
        #priority=wd.find_element(By.CSS_SELECTOR,"div.ant-row.ant-form-item.has(> label[for='Task Form (update)_priority'])*::text")
        #priority=None 
        task_activities=wd.find_elements(By.CSS_SELECTOR,"div.ant-timeline-item-content div.ant-row.ant-row-middle[role='row']")
        activities=[]
        time_stamps=[]
        try:bounties=wd.find_element(By.CSS_SELECTOR,"span.ant-tag[style='background-color: white; color: black; min-width: 0px; overflow: hidden; text-overflow: ellipsis;']").get_attribute("textContent")
        except: bounties=None
      
         
        try:reviwers=wd.find_element(By.CSS_SELECTOR,"div.ant-row.ant-form-item:has( label[for='Task Form (update)_ownerIds'])").find_element(By.CSS_SELECTOR,"div:nth-child(2)").get_attribute("textContent")   
        except:reviwers=None 
        try:priority=wd.find_element(By.CSS_SELECTOR,"div.ant-row.ant-form-item:has( label[for='Task Form (update)_priority'])").find_element(By.CSS_SELECTOR,"div:nth-child(2)").get_attribute("textContent")
        except:priority=None  
        try:activity_comments=[i.text for i in wd.find_elements(By.CSS_SELECTOR,"div.ant-timeline-item-content div.ProseMirror[role='textbox'][contenteditable='false'] > p")]
        except:activity_comments=None
        for activity in task_activities:
                                 #print(activity.text+"/")
            try:activities.append(activity.find_element(By.CLASS_NAME,"ant-typography").get_attribute("textContent"))
            except:pass
            try:time_stamps.append(activity.find_element(By.CLASS_NAME,"ant-typography-secondary").get_attribute("textContent"))
            except:pass
        try:subtasks=[i.get_attribute("textContent") for i in wd.find_elements(By.CSS_SELECTOR,"td.ant-table-cell.w-full > div.ant-typography")]
        except:subtasks=None 
        try:subtasks_id=[i.get_attribute("data-row-key") for i in wd.find_elements(By.CSS_SELECTOR,"tbody.ant-table-tbody > tr[data-row-key][index]")]
        except:subtasks_id=None
        try: tags=[i.find_element(By.CSS_SELECTOR,"span.ant-tag").get_attribute("textContent") for i in wd.find_elements(By.CSS_SELECTOR,"div.ant-select-selection-overflow-item > span")]
        except:tags=None
        tasks.append({"activity_comments":activity_comments,"link":self.url,"labels":labels,"tags":tags,"subtasks_id":subtasks_id,"reviewers":reviwers,"DAO":DAO["DAO"],"time_stamps":time_stamps,"activities": activities,"subtasks":subtasks,"priority":priority,"task_title":task_title,"task_description":task_description,"subtasks":subtasks,"bounties":bounties})
        wd.close()

In [ ]:
data.head(10)